In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import Dataset

In [43]:
# Load the CSV file
csv_file = './../3. Cleaned Data/20K.csv'
df = pd.read_csv(csv_file)

In [44]:
df.head()

,Unnamed: 0,original,word_count_original,corrected_fs,word_count_corrected_fs,word_count_diff,word_count_ratio,o_pos_tags,o_num_verbs,o_num_nouns,...,c_num_modal,num_verbs_diff,num_nouns_diff,num_adjs_diff,num_adv_diff,num_pronoun_diff,num_preposition_diff,num_conjunction_diff,num_article_diff,num_modal_diff
0,0,I will appreciate it if you correct my sentences.,9,I would appreciate it if you could correct my ...,10,1,1.111111,"[('I', 'PRP'), ('will', 'MD'), ('appreciate', ...",2,1,...,2,0,0,0,0,0,0,0,0,1
1,1,It's been getting colder these days here in Ja...,9,It's been getting colder lately here in Japan.,8,-1,0.888889,"[('It', 'PRP'), (""'s"", 'VBZ'), ('been', 'VBN')...",3,3,...,0,0,-2,0,2,0,0,0,-1,0
2,2,The summer weather in Japan is not agreeable t...,16,I find Japan's summer weather disagreeable bec...,13,-3,0.812500,"[('The', 'DT'), ('summer', 'NN'), ('weather', ...",1,5,...,0,0,0,0,-1,0,0,0,-1,0
3,3,"So, as the winter is coming, I'm getting to fe...",11,"So, as the winter is coming, I'm starting to f...",11,0,1.000000,"[('So', 'RB'), (',', ','), ('as', 'IN'), ('the...",5,1,...,0,0,0,0,0,0,0,0,0,0
4,4,It is the very exciting season.,6,It is a very exciting season.,6,0,1.000000,"[('It', 'PRP'), ('is', 'VBZ'), ('the', 'DT'), ...",1,1,...,0,0,0,0,0,0,0,0,0,0


In [45]:
# Change the column names in the dataframe
df.rename(columns = {'corrected_fs':'corrected'}, inplace = True)

In [46]:
# Split the dataset into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert the train and validation DataFrames to Hugging Face's Dataset instances
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [47]:
# Chose the model
model_name = 'gpt2'

In [48]:
# Assign cuda to the device to use for training
if torch.cuda.is_available(): 
 dev = "cuda:0" 
 print("This model will run on CUDA")
elif  torch.backends.mps.is_available(): 
 dev = "mps:0"
 print("This model will run on MPS")
else:
 dev = "cpu" 
 print("This model will run on CPU")
device = torch.device(dev) 

print(device)

This model will run on CUDA
cuda:0


In [49]:
# Load the tokenizer and the model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
config = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, config=config).to(device)

In [50]:
# Ensure that the tokenizer uses the same special tokens as GPT-2
tokenizer.pad_token = tokenizer.eos_token

# Function to tokenize and format input-output pairs
def tokenize_function(examples):
    inputs = [f"input: {orig} output: {corr}" for orig, corr in zip(examples["original"], examples["corrected"])]
    return tokenizer(inputs, padding=True, truncation=True, max_length=512, return_tensors='pt')

In [51]:
# Tokenize the train and validation data
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['original', 'corrected'])
val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=['original', 'corrected'])

KeyboardInterrupt: 

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_steps=500,
    save_steps=500,
    warmup_steps=200,
    logging_dir="logs",
    evaluation_strategy="steps",
    logging_steps=100,
)

In [ ]:
# Define a custom loss function to focus on the "output" tokens
def custom_loss_function(outputs, labels):
    shift_logits = outputs.logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    loss = torch.nn.CrossEntropyLoss()(shift_logits, shift_labels)
    return loss

In [ ]:
# Define a custom Trainer class that inherits from the original Trainer
class CustomTrainer(Trainer):
    
    # Override the compute_loss method to use a custom loss function
    def compute_loss(self, model, inputs, return_outputs=False):
        # Get the labels from the inputs dictionary and remove them from the inputs
        labels = inputs.pop("labels")
        
        # Get the model outputs by passing the inputs to the model
        outputs = model(**inputs)
        
        # Extract the logits from the model outputs
        logits = outputs.logits
        
        # Get the correct dimensions for the shift_labels tensor
        shift_labels = labels[..., 1:].reshape(-1)

        # Reshape the shift_logits tensor to align with the dimensions of the shift_labels tensor
        shift_logits = logits[..., :-1, :].reshape(-1, logits.size(-1))

        loss = torch.nn.CrossEntropyLoss()(shift_logits, shift_labels)

        if return_outputs:
            return loss, outputs
        
        # Otherwise, just return the loss
        return loss

# Create the custom Trainer with the custom loss function
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    
    # Use the DataCollatorForLanguageModeling to handle the data collation
    # Set mlm=False, as we are not using masked language modeling
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

In [ ]:
print(model)
print(device)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Save the trained model and tokenizer
output_dir = "7. Models/20K/"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Then compress with this command: tar czvf trained_model.tar.gz trained_model/
# Upload to git/drive


In [ ]:
# Load trained model
output_dir = "7. Models/20K/"
model = GPT2LMHeadModel.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
